In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
with open('CoRef-TheDaVinciCode.txt', 'r') as f:
    data = f.read()
    data = data.replace("\n", " ")
    data = data.replace("\t", " ")
    data = data.replace("\r", " ")
    data = data.replace("  ", " ")
    data = data.replace("   ", " ")
    data = data.replace("    ", " ")
    data = data.replace("     ", " ")
    ' '.join(data.split())
    data = re.sub('/\s+/', ' ', data)
f.close()

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [5]:
# Remove new line characters
data = re.sub('\s+', ' ', data)

# Remove distracting single quotes
data = re.sub("\'", "", data)

In [7]:
def sent_to_words(sentences):
    yield(gensim.utils.simple_preprocess(str(sentences), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['chapter', 'the', 'da', 'vinci', 'code', 'dan', 'brown', 'prologue', 'louvre', 'museum', 'paris', 'renowned', 'curator', 'jacques', 'sauniere', 'staggered', 'through', 'the', 'vaulted', 'archway', 'of', 'louvre', 'museums', 'grand', 'gallery', 'renowned', 'curator', 'jacques', 'sauniere', 'lunged', 'for', 'the', 'nearest', 'painting', 'he', 'could', 'see', 'caravaggio', 'grabbing', 'the', 'gilded', 'frame', 'renowned', 'curator', 'jacques', 'sauniere', 'heaved', 'the', 'nearest', 'painting', 'he', 'could', 'see', 'caravaggio', 'toward', 'renowned', 'curator', 'jacques', 'sauniere', 'until', 'the', 'nearest', 'painting', 'he', 'could', 'see', 'caravaggio', 'tore', 'from', 'the', 'wall', 'and', 'renowned', 'curator', 'jacques', 'sauniere', 'collapsed', 'backward', 'in', 'heap', 'beneath', 'the', 'canvas', 'as', 'renowned', 'curator', 'jacques', 'sauniere', 'had', 'anticipated', 'thundering', 'iron', 'gate', 'fell', 'nearby', 'barricading', 'the', 'entrance', 'to', 'the', 'suite', 'the'

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['chapter', 'the', 'da_vinci', 'code', 'dan', 'brown', 'prologue', 'louvre', 'museum', 'paris', 'renowned_curator_jacques_sauniere', 'staggered', 'through', 'the', 'vaulted_archway', 'of', 'louvre', 'museums', 'grand_gallery', 'renowned_curator_jacques_sauniere', 'lunged', 'for', 'the', 'nearest', 'painting', 'he', 'could', 'see', 'caravaggio', 'grabbing', 'the', 'gilded', 'frame', 'renowned_curator_jacques_sauniere', 'heaved', 'the', 'nearest', 'painting', 'he', 'could', 'see', 'caravaggio', 'toward', 'renowned_curator_jacques_sauniere', 'until', 'the', 'nearest', 'painting', 'he', 'could', 'see', 'caravaggio', 'tore', 'from', 'the', 'wall', 'and', 'renowned_curator_jacques_sauniere', 'collapsed', 'backward', 'in', 'heap', 'beneath', 'the', 'canvas', 'as', 'renowned_curator_jacques_sauniere', 'had', 'anticipated', 'thundering', 'iron', 'gate', 'fell', 'nearby', 'barricading', 'the', 'entrance', 'to', 'the', 'suite', 'the', 'parquet', 'floor', 'shook', 'far', 'off', 'an', 'alarm', 'beg

In [9]:
nlp = spacy.load("en_core_web_sm")

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [13]:
from nltk.stem import WordNetLemmatizer
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams

[['chapter',
  'da_vinci',
  'code',
  'dan',
  'brown',
  'prologue',
  'louvre',
  'museum',
  'paris',
  'renowned_curator',
  'jacques_sauniere',
  'staggered',
  'vaulted_archway',
  'louvre',
  'museums',
  'grand_gallery',
  'renowned_curator',
  'jacques_sauniere',
  'lunged',
  'nearest',
  'painting',
  'could',
  'see',
  'caravaggio',
  'grabbing',
  'gilded',
  'frame',
  'renowned_curator',
  'jacques_sauniere',
  'heaved',
  'nearest',
  'painting',
  'could',
  'see',
  'caravaggio',
  'toward',
  'renowned_curator',
  'jacques_sauniere',
  'nearest',
  'painting',
  'could',
  'see',
  'caravaggio',
  'tore',
  'wall',
  'renowned_curator',
  'jacques_sauniere',
  'collapsed',
  'backward',
  'heap',
  'beneath',
  'canvas',
  'renowned_curator',
  'jacques_sauniere',
  'anticipated',
  'thundering',
  'iron',
  'gate',
  'fell',
  'nearby',
  'barricading',
  'entrance',
  'suite',
  'parquet',
  'floor',
  'shook',
  'far',
  'alarm',
  'began',
  'ring',
  'renowned

In [24]:
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
data_lemmatized = [lemmatizer.lemmatize(x) for y in data_words_bigrams for x in y]
# lemmatize data_words_bigrams
# data_lemmatized = lemmatizer.lemmatize(data_words_bigrams)

print(data_lemmatized)

['chapter', 'da_vinci', 'code', 'dan', 'brown', 'prologue', 'louvre', 'museum', 'paris', 'renowned_curator', 'jacques_sauniere', 'staggered', 'vaulted_archway', 'louvre', 'museum', 'grand_gallery', 'renowned_curator', 'jacques_sauniere', 'lunged', 'nearest', 'painting', 'could', 'see', 'caravaggio', 'grabbing', 'gilded', 'frame', 'renowned_curator', 'jacques_sauniere', 'heaved', 'nearest', 'painting', 'could', 'see', 'caravaggio', 'toward', 'renowned_curator', 'jacques_sauniere', 'nearest', 'painting', 'could', 'see', 'caravaggio', 'tore', 'wall', 'renowned_curator', 'jacques_sauniere', 'collapsed', 'backward', 'heap', 'beneath', 'canvas', 'renowned_curator', 'jacques_sauniere', 'anticipated', 'thundering', 'iron', 'gate', 'fell', 'nearby', 'barricading', 'entrance', 'suite', 'parquet', 'floor', 'shook', 'far', 'alarm', 'began', 'ring', 'renowned_curator', 'jacques_sauniere', 'lay', 'moment', 'gasping', 'breath', 'taking', 'stock', 'still', 'alive', 'renowned_curator', 'jacques_saunier

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kristianvankuijk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
data_lemmatized = [data_lemmatized]
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 3), (1, 1), (2, 4), (3, 1), (4, 7), (5, 1), (6, 1), (7, 1), (8, 7), (9, 2), (10, 14), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 4), (18, 1), (19, 5), (20, 1), (21, 4), (22, 2), (23, 6), (24, 1), (25, 2), (26, 1), (27, 1), (28, 3), (29, 2), (30, 5), (31, 2), (32, 1), (33, 3), (34, 1), (35, 1), (36, 13), (37, 1), (38, 6), (39, 6), (40, 4), (41, 15), (42, 4), (43, 2), (44, 2), (45, 6), (46, 1), (47, 2), (48, 7), (49, 1), (50, 20), (51, 1), (52, 6), (53, 1), (54, 6), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 5), (61, 1), (62, 12), (63, 6), (64, 2), (65, 1), (66, 1), (67, 2), (68, 18), (69, 24), (70, 30), (71, 1), (72, 1), (73, 3), (74, 4), (75, 4), (76, 1), (77, 5), (78, 4), (79, 3), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 4), (87, 1), (88, 121), (89, 20), (90, 1), (91, 2), (92, 16), (93, 3), (94, 8), (95, 3), (96, 3), (97, 1), (98, 56), (99, 2), (100, 2), (101, 4), (102, 13), (103, 20), (104, 6), (105, 4), (106, 1), (107, 1), (108, 2), (10

In [30]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aah', 3),
  ('abandon', 1),
  ('abandoned', 4),
  ('abbe', 1),
  ('abbey', 7),
  ('abbreviated', 1),
  ('abdomen', 1),
  ('abided', 1),
  ('ability', 7),
  ('ablaze', 2),
  ('able', 14),
  ('aboard', 1),
  ('abolish', 1),
  ('abounded', 1),
  ('abracadabra', 1),
  ('abreast', 2),
  ('abrupt', 1),
  ('abruptly', 4),
  ('absenle', 1),
  ('absent', 5),
  ('absente', 1),
  ('absently', 4),
  ('absolute', 2),
  ('absolutely', 6),
  ('absolution', 1),
  ('absorb', 2),
  ('abstinence', 1),
  ('abstractedly', 1),
  ('absurd', 3),
  ('absurdity', 2),
  ('absurdly', 5),
  ('abu', 2),
  ('abundance', 1),
  ('abuse', 3),
  ('abused', 1),
  ('abyss', 1),
  ('academic', 13),
  ('accelerate', 1),
  ('accelerated', 6),
  ('accelerating', 6),
  ('accelerator', 4),
  ('accent', 15),
  ('accented', 4),
  ('accentuated', 2),
  ('accentuating', 2),
  ('accept', 6),
  ('acceptable', 1),
  ('acceptance', 2),
  ('accepted', 7),
  ('accepting', 1),
  ('access', 20),
  ('accessed', 1),
  ('accessible', 6),


In [31]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [32]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.002*"langdon" + 0.001*"sophie" + 0.001*"robert" + 0.001*"silas" + '
  '0.001*"langdons" + 0.001*"said" + 0.001*"neveu" + 0.000*"one" + '
  '0.000*"teabings" + 0.000*"sophies"'),
 (1,
  '0.002*"langdon" + 0.001*"sophie" + 0.001*"robert" + 0.001*"silas" + '
  '0.000*"grandfather" + 0.000*"teabing" + 0.000*"one" + 0.000*"langdons" + '
  '0.000*"sophies" + 0.000*"said"'),
 (2,
  '0.003*"langdon" + 0.002*"sophie" + 0.001*"robert" + 0.001*"silas" + '
  '0.001*"teabings" + 0.001*"said" + 0.001*"grandfather" + 0.001*"langdons" + '
  '0.001*"teabing" + 0.001*"sophies"'),
 (3,
  '0.003*"langdon" + 0.002*"sophie" + 0.001*"robert" + 0.001*"langdons" + '
  '0.001*"teabing" + 0.001*"grandfather" + 0.001*"silas" + 0.001*"sophies" + '
  '0.001*"said" + 0.001*"teabings"'),
 (4,
  '0.004*"langdon" + 0.003*"sophie" + 0.001*"robert" + 0.001*"sophies" + '
  '0.001*"silas" + 0.001*"langdons" + 0.001*"neveu" + 0.001*"teabings" + '
  '0.001*"grandfather" + 0.001*"one"'),
 (5,
  '0.001*"sophie" + 0.0

In [34]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.199843 -0.000001       1        1  99.999704
19    -0.059733  0.000025       2        1   0.000018
3      0.006411  0.000100       3        1   0.000018
16     0.006478 -0.000086       4        1   0.000018
18     0.001843  0.000040       5        1   0.000017
4     -0.004869  0.000179       6        1   0.000015
14     0.036715  0.000065       7        1   0.000015
5      0.035571 -0.000073       8        1   0.000015
15     0.032167  0.000116       9        1   0.000015
6      0.025288 -0.000137      10        1   0.000015
13     0.023158 -0.000056      11        1   0.000015
1      0.021802  0.000016      12        1   0.000015
7      0.021003 -0.000159      13        1   0.000015
17     0.019960  0.000215      14        1   0.000015
9     -0.006028  0.000152      15        1   0.000015
10     0.018414 -0.000149      16        1   0.000015
12     0.017434  0.000047      17        1   0.000015
0      0.013318 -0.000109      18        1   0.000015
2      0.007734  0.000129      19        1   0.000015
11    -0.016822 -0.000313      20        1   0.000014, topic_info=           Term         Freq        Total Category  logprob  loglift
5114    langdon  2892.000000  2892.000000  Default  30.0000  30.0000
8578     sophie  2280.000000  2280.000000  Default  29.0000  29.0000
7739     robert  1090.000000  1090.000000  Default  28.0000  28.0000
8323      silas   686.000000   686.000000  Default  27.0000  27.0000
5121   langdons   610.000000   610.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1504     church     0.000007   312.182445  Topic20  -7.5210  -1.8055
9646        two     0.000006   196.965441  Topic20  -7.6341  -1.4581
10393    zurich     0.000007   221.211261  Topic20  -7.6133  -1.5534
3282        eye     0.000007   244.267992  Topic20  -7.5964  -1.6355
5008   keystone     0.000007   218.121046  Topic20  -7.6280  -1.5540

[1361 rows x 6 columns], token_table=       Topic      Freq               Term
term                                     
162        1  1.000165              agent
310        1  1.000105       andre_vernet
665        1  0.998675               back
865        1  0.999111         bezu_fache
899        1  0.999665  bishop_aringarosa
1504       1  0.999416             church
1991       1  0.999057              could
2417       1  0.998097    depository_bank
2704       1  0.998762               door
3146       1  0.999360               even
3282       1  0.998903                eye
3300       1  0.999441              fache
3407       1  0.998601               felt
3560       1  0.998511              floor
3996       1  0.999284        grandfather
4152       1  0.999734               hand
4381       1  0.999199         holy_grail
5004       1  0.998107                key
5008       1  0.999445           keystone
5058       1  1.000180               know
5114       1  1.000028            langdon
5121       1  0.999447           langdons
5230       1  1.001093              leigh
5287       1  0.999481               like
5379       1  0.999596               long
5393       1  0.999986             looked
5525       1  0.998904                man
6031       1  0.999249              neveu
6248       1  1.000047                one
7009       1  0.998960             priory
7694       1  1.000008              right
7739       1  0.999806             robert
7886       1  1.000743               said
8062       1  0.999306             secret
8077       1  0.999941                see
8323       1  0.999683              silas
8326       1  0.997943             silass
8372       1  0.999120          sir_leigh
8578       1  0.999902             sophie
8583       1  1.000477            sophies
9159       1  1.000808            teabing
9164       1  0.999776           teabings
9178       1  0.998714            teacher
9311       1 